**Information on Forest**

Individualized approach for metrics on deforestation occurrences in global monitoring alerts - "In Off, I am doing more alerts"

In [1]:
import ee
import geemap
import datetime

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
# Create Filter alert date
max_pixel_age = 4.5 #years
oldest_pixel_date = ee.Date(datetime.datetime.now()).advance(-max_pixel_age, 'year') # alerts 4 years and half old (54 months)
reference_date = ee.Date('2014-12-31') # RADD reference date
outdated_trashold = ee.Number(oldest_pixel_date.difference(reference_date, 'days')).round()

In [4]:
# Area of interest
#aoi = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0').filter('ADM0_NAME == "Brazil"').geometry()
#aoi = ee.FeatureCollection('FAO/GAUL/2015/level1').filter('ADM1_NAME == "Amazonas"').geometry()
#aoi = ee.FeatureCollection('FAO/GAUL/2015/level2').filter('ADM2_NAME == "Porto Velho"').geometry()
aoi = ee.Geometry.Polygon([-64, -9.5, -63.7, -9.5, -63.7, -9.7, -64, -9.7])

# WUR Radar Alerts https://www.wur.nl/en/research-results/chair-groups/environmental-sciences/laboratory-of-geo-information-science-and-remote-sensing/research/sensing-measuring/radd-forest-disturbance-alert.htm
radd_alert = (
    ee.Image(
        ee.ImageCollection('projects/radar-wur/raddalert/v1').filterMetadata('layer','contains','alert')
        .filterMetadata('geography','equals','sa')
        .sort('system:time_end', False)
        .first()
    )
    .clip(aoi)
    .select('Date')
)

# GLAD Optical Alerts https://glad-forest-alert.appspot.com/
#glad_alert = ee.Image('projects/glad/alert/UpdResult/10_20_SA').select('alertDate23').clip(aoi)

In [5]:
pixelTreshold = 100 # 100 pixels ~ 1 hectare
confidence = radd_alert.divide(10000).int() # Confidence Raster
radd_alert_date = radd_alert.subtract(confidence.multiply(10000)) #Update pixel values to keep only date
mascara_old = radd_alert_date.gt(outdated_trashold) # Mask old deforestation
mascara_old = mascara_old.updateMask(mascara_old)
mascara_small = mascara_old.connectedPixelCount(1024).gt(pixelTreshold)
mascara_small = mascara_small.updateMask(mascara_small)
radd_alert_id = mascara_small.connectedComponents(ee.Kernel.square(1), 1024)

In [6]:
deforestation = radd_alert_id.select('labels').reduceToVectors(bestEffort = True)

In [41]:
minMaxDateDeforestation = radd_alert_date.reduceRegions(
    collection=deforestation,
    reducer=ee.Reducer.minMax(),
    scale=10,  # meters
    crs='EPSG:4326',
)

In [42]:
def metrics_generation(feature):
  max_d = feature.get('max')
  min_d = feature.get('min')
  area_ha = ee.Number(feature.get('count')).divide(1000)
  persistence = ee.Number(max_d).subtract(min_d)
  isdo = ee.Number(area_ha).divide(persistence)  
  min_date = ee.Date('2014-12-31').advance(min_d, 'days').format("YYYY-MM-dd")
  max_date = ee.Date('2014-12-31').advance(max_d, 'days').format("YYYY-MM-dd")
  return feature.set('persistence', persistence).set('area_ha', area_ha).set('persistence', persistence).set('isdo', isdo).set('min_date', min_date).set('max_date', max_date)

minMaxDateDeforestation = minMaxDateDeforestation.map(metrics_generation)

In [45]:
def age_generation(feature):
  md = feature.get('max_date')
  age_weeks = ee.Date(datetime.datetime.now()).difference(ee.Date(md), 'week').round()
  # Add the difference as a new property
  return feature.set('age_weeks', age_weeks)

minMaxDateDeforestation = minMaxDateDeforestation.map(age_generation)

In [44]:
geemap.ee_export_vector_to_drive(
    minMaxDateDeforestation, description="teste", fileFormat='GeoJSON', folder="GEE_exports"
)

Exporting teste... Please check the Task Manager from the JavaScript Code Editor.


In [50]:
geemap.ee_export_image_to_drive(
    radd_alert_date.toUint16(), description='radd_alert_date', folder='GEE_exports', region=aoi, scale=10
)

In [46]:
m = geemap.Map(height=800)

In [47]:
m.addLayer(radd_alert_date.mask(radd_alert_date), {}, 'radd_alert_date')
m.addLayer(radd_alert.mask(radd_alert), {}, 'radd_alert')
m.addLayer(minMaxDateDeforestation, {}, 'Deforestation')

In [48]:
m.center_object(aoi)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…